### Import Library

In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib
import numpy as np
import textwrap
import datetime

# inisialisasi now sebagai nama file 
now = datetime.datetime.today().strftime('%d_%m_%Y')

### Proses Data Dosen

In [26]:
dosen_2023 = pd.read_excel(f'data//2023.xlsx')
dosen_2023 = dosen_2023[['NIP', 'NAMA (BERSERA GELAR)', 'JK', 'PEND', 'GOL', 'JABFUNG', 'STATUS PEGAWAI',  
                        'FAKULTAS']]
dosen_2023 = dosen_2023.rename(columns = {'NIP': 'nip', 'NAMA (BERSERA GELAR)': 'nama', 'STATUS PEGAWAI': 'status_pegawai',
                                          'JK': 'gender', 'PEND': 'pend_terakhir', 'GOL': 'gol', 'JABFUNG': 'jabfung',
                                          'FAKULTAS': 'fakultas'})

# fungsi untuk ubah gender menjadi Laki-laki dan Perempuan
def change_gender(col):
    if col == 'L':
        col = 'Laki-laki'
    elif col == 'P':
        col = 'Perempuan'
    return col

# fungsi untuk replace beberapa data nip yang tidak sesuai format
def replace_data_2023(df):
    df['nip'].iat[1463] = '020150819560721101'
    df['nip'].iat[1464] = '020150819590722201'
    df['nip'].iat[1465] = '020190319571017101'
    df['nip'].iat[1466] = '020190319650315201'
    
    return df

# fungsi untuk extract usia dosen
def extract_usia(df):
    
    # mark sebagai penanda penghitung usia
    mark = 0
    batas = 1109
    mark = 1
        
    # proses data bagian pertama
    df1 = df[:batas].copy()
    df1['nip'] = df1['nip'].astype(str)
    df1['usia'] = df1['nip'].str.slice(0, -10)
    df1['usia'] = pd.to_datetime(df1['usia'], format='%Y%m%d')
    today = pd.Timestamp.now()
    
    # kondisi jika mark == 1 maka tahun now diset sebagai tahun sekarang
    if mark == 1:
        now = pd.Timestamp('now')
    # kondisi jika mark != 1 maka tahun pengurang disesuaikan dengan counter
    else:
        now = today - pd.Timedelta(days=365 * counter)
    # extract usia
    df1['usia'] = (now.year - df1['usia'].dt.year) - ((now.month - df1['usia'].dt.month) < 0)
    
    # proses data bagian kedua
    df2 = df[batas:].copy()
    df2['nip'] = df2['nip'].astype(str)
    df2['usia'] = df2['nip'].str.slice(7, -3)
    df2['usia'] = pd.to_datetime(df2['usia'], format='%Y%m%d')
    df2['usia'] = (now.year - df2['usia'].dt.year) - ((now.month - df2['usia'].dt.month) < 0)

    # sambung data bagian pertama dan kedua menjadi satu 
    df = pd.concat([df1, df2], ignore_index=True)
    return df

def extract_substr(text, mode):
    string = text.split(',', 1)

    string = string[0]    
    string = string.replace('Dipl. ', '')
    string = string.replace('Dr. ', '')
    string = string.replace('Eng. ', '')
    string = string.replace('dr. ', '')
    string = string.replace('phil. ', '')
    string = string.replace('Dra. ', '')
    string = string.replace('Drs. ', '')
    string = string.replace('H. ', '')
    string = string.replace('Hj. ', '')
    string = string.replace('Ir. ', '')
    string = string.replace('Prof. ', '')
    string = string.replace('paed. ', '')
    string = string.replace('Phil. ', '')
    
    return string

# apply fungsi pada kolom nama kemudian buat kolom baru 'Nama Tanpa Gelar'
dosen_2023['Nama Tanpa Gelar'] = dosen_2023['nama'].apply(lambda x: extract_substr(x, 0))

# apply fungsi untuk menghasilkan format data yang sesuai
dosen_2023['gender'] = dosen_2023['gender'].apply(lambda x: change_gender(x))
dosen_2023 = replace_data_2023(dosen_2023)
dosen_2023 = extract_usia(dosen_2023)
dosen_2023.head()

,nip,nama,gender,pend_terakhir,gol,jabfung,status_pegawai,fakultas,Nama Tanpa Gelar,usia
0,196202081986011002,"Prof. Dr. H. M. Solehuddin, M.Pd., M.A.",Laki-laki,S3,IV/c,Guru Besar,PNS,FIP,M. Solehuddin,61
1,195306031980021003,"Prof. Dr. H. Mohammad Ali, M.Pd., M.A.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,Mohammad Ali,69
2,195404021980112001,"Prof. Dr. Hj. Ihat Hatimah, M.Pd.",Perempuan,S3,IV/e,Guru Besar,PNS,FIP,Ihat Hatimah,68
3,195501011981011001,"Prof. Dr. H. Achmad Hufad, M.Ed.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,Achmad Hufad,68
4,196111091987031001,"Prof. Dr. H. Mustofa Kamil, M.Pd.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,Mustofa Kamil,61


In [18]:
dosen_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1566 entries, 0 to 1565
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   nip             1566 non-null   object
 1   nama            1566 non-null   object
 2   status_pegawai  1566 non-null   object
 3   tgl_lahir       1438 non-null   object
 4   gender          1566 non-null   object
 5   gol             1549 non-null   object
 6   jabfung         1566 non-null   object
 7   fakultas        1566 non-null   object
 8   usia            1566 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 110.2+ KB


### Proses Data SINTA

In [29]:
df_sinta = pd.read_csv('data//dosen_sinta_10-03-2023.csv', encoding='utf-8')
df_map = pd.read_csv('data//dosen_mapping_26_1_2023.csv', encoding='utf-8')

# df_sinta['Nama Tanpa Gelar'] = df_sinta['nama'].apply(lambda x: extract_substr(x, 0))

df_map.head()

,nama,sinta_id,NIP
0,"Prof. Dr. H. M. Solehuddin, M.Pd., M.A.",5995348.0,196202081986011002
1,"Prof. Dr. H. Mohammad Ali, M.Pd., M.A.",5980905.0,195306031980021003
2,"Prof. Dr. Hj. Ihat Hatimah, M.Pd.",5979054.0,195404021980112001
3,"Prof. Dr. H. Achmad Hufad, M.Ed.",5977826.0,195501011981011001
4,"Prof. Dr. H. Mustofa Kamil, M.Pd.",5994180.0,196111091987031001
